In [8]:
import pandas as pd
import sqlite3
import os

In [9]:
# 1. Настройки путей
# Поднимаемся на уровень выше, так как ноутбук в папке notebooks
base_path = '../data/raw'
db_path = '../data/processed/retail_rocket.db'

In [10]:
# 2. Загрузка Events (самый важный файл)
print("Загружаем events.csv...")
events_df = pd.read_csv(os.path.join(base_path, 'events.csv'))

# ПРОВЕРКА ДАННЫХ
print(f"Размер датасета: {events_df.shape}")
print("\nПример данных:")
display(events_df.head())

Загружаем events.csv...
Размер датасета: (2756101, 5)

Пример данных:


,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [11]:
# 3. Предобработка (Data Cleaning)
# Retail Rocket хранит время в миллисекундах. Конвертируем в datetime сразу,
# чтобы в SQL было удобно работать.
events_df['event_time'] = pd.to_datetime(events_df['timestamp'], unit='ms')

print("\nДанные после обработки времени:")
display(events_df.head())


Данные после обработки времени:


,timestamp,visitorid,event,itemid,transactionid,event_time
0,1433221332117,257597,view,355908,NaN,2015-06-02 05:02:12.117
1,1433224214164,992329,view,248676,NaN,2015-06-02 05:50:14.164
2,1433221999827,111016,view,318965,NaN,2015-06-02 05:13:19.827
3,1433221955914,483717,view,253185,NaN,2015-06-02 05:12:35.914
4,1433221337106,951259,view,367447,NaN,2015-06-02 05:02:17.106


In [12]:
# 4. Загрузка в SQLite
print(f"\nСохраняем в базу данных: {db_path}...")

# Создаем коннекшн
conn = sqlite3.connect(db_path)

# chunksize помогает, если данных много, чтобы не забить память
events_df.to_sql('events', conn, if_exists='replace', index=False, chunksize=10000)


Сохраняем в базу данных: ../data/processed/retail_rocket.db...


2756101

In [13]:
# 5. (Опционально) Загрузим дерево категорий, если пригодится позже
cat_df = pd.read_csv(os.path.join(base_path, 'category_tree.csv'))
cat_df.to_sql('category_tree', conn, if_exists='replace', index=False)

print("Готово! Данные успешно загружены в БД.")
conn.close()

Готово! Данные успешно загружены в БД.


In [14]:
# Тестовый SQL запрос
conn = sqlite3.connect('../data/processed/retail_rocket.db')

query = """
SELECT 
    event, 
    COUNT(*) as count 
FROM events 
GROUP BY event
ORDER BY count DESC
"""

df_test = pd.read_sql(query, conn)
display(df_test)
conn.close()

,event,count
0,view,2664312
1,addtocart,69332
2,transaction,22457


In [15]:
# SQL скрипт для сессионизации
# Мы используем колонку 'timestamp' (миллисекунды), которая осталась в данных
# 30 минут = 30 * 60 * 1000 = 1,800,000 миллисекунд

sessionization_query = """
WITH events_sorted AS (
    -- 1. Сортируем события и находим время предыдущего события
    SELECT 
        visitorid,
        event,
        timestamp,
        event_time,
        LAG(timestamp) OVER (PARTITION BY visitorid ORDER BY timestamp) as prev_timestamp
    FROM events
),
session_flags AS (
    -- 2. Размечаем начало новой сессии
    SELECT 
        *,
        CASE 
            WHEN prev_timestamp IS NULL THEN 1 -- Первое событие пользователя
            WHEN (timestamp - prev_timestamp) > 1800000 THEN 1 -- Прошло > 30 мин
            ELSE 0 
        END as is_new_session
    FROM events_sorted
),
session_ids AS (
    -- 3. Генерируем ID сессии через нарастающую сумму
    SELECT 
        *,
        SUM(is_new_session) OVER (PARTITION BY visitorid ORDER BY timestamp) as user_session_index
    FROM session_flags
)
-- 4. Агрегируем данные в готовую витрину сессий
SELECT 
    visitorid || '_' || user_session_index as session_id, -- Уникальный ID (строка)
    visitorid,
    MIN(event_time) as session_start,
    MAX(event_time) as session_end,
    COUNT(*) as total_events,
    -- Считаем конкретные действия внутри сессии (One-Hot Encoding в SQL)
    SUM(CASE WHEN event = 'view' THEN 1 ELSE 0 END) as view_count,
    SUM(CASE WHEN event = 'addtocart' THEN 1 ELSE 0 END) as cart_count,
    SUM(CASE WHEN event = 'transaction' THEN 1 ELSE 0 END) as transaction_count,
    -- Флаг покупки (была ли покупка в этой сессии?)
    MAX(CASE WHEN event = 'transaction' THEN 1 ELSE 0 END) as is_purchase
FROM session_ids
GROUP BY 1, 2
"""

print("Выполняем сессионизацию (это может занять пару минут)...")

# Подключаемся и выполняем
conn = sqlite3.connect('../data/processed/retail_rocket.db')

# Читаем результат запроса в DataFrame
df_sessions = pd.read_sql(sessionization_query, conn)

# Сохраняем обратно в SQL как готовую витрину
df_sessions.to_sql('sessions', conn, if_exists='replace', index=False)

print(f"Готово! Создана таблица 'sessions'.")
print(f"Всего сессий: {len(df_sessions)}")
display(df_sessions.head())

conn.close()

Выполняем сессионизацию (это может занять пару минут)...
Готово! Создана таблица 'sessions'.
Всего сессий: 1761675


,session_id,visitorid,session_start,session_end,total_events,view_count,cart_count,transaction_count,is_purchase
0,0_1,0,2015-09-11 20:49:49.439000,2015-09-11 20:55:17.175000,3,3,0,0,0
1,1000000_1,1000000,2015-06-05 18:16:10.629000,2015-06-05 18:16:10.629000,1,1,0,0,0
2,1000001_1,1000001,2015-07-07 18:12:14.953000,2015-07-07 18:12:14.953000,1,1,0,0,0
3,1000001_2,1000001,2015-07-24 20:18:15.303000,2015-07-24 20:35:57.029000,3,3,0,0,0
4,1000001_3,1000001,2015-07-29 20:38:29.170000,2015-07-29 20:38:29.170000,1,1,0,0,0


In [ ]:
display(df_sessions[(df_sessions['cart_count'] > 0) & (df_sessions['is_purchase'] == 0)])

print("Кол-во брошенных корзин:")
print(len(df_sessions[(df_sessions['cart_count'] > 0) & (df_sessions['is_purchase'] == 0)]))

,session_id,visitorid,session_start,session_end,total_events,view_count,cart_count,transaction_count,is_purchase
107,100007_1,100007,2015-08-18 18:27:15.860000,2015-08-18 18:28:07.752000,5,3,2,0,0
143,1000103_1,1000103,2015-05-27 17:46:30.856000,2015-05-27 18:04:36.274000,4,3,1,0,0
234,1000169_1,1000169,2015-08-02 04:26:42.679000,2015-08-02 04:27:49.561000,4,3,1,0,0
455,1000306_13,1000306,2015-06-24 20:03:13.645000,2015-06-24 21:05:57.393000,6,5,1,0,0
456,1000306_14,1000306,2015-06-25 18:54:52.524000,2015-06-25 18:54:52.524000,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1761527,999895_1,999895,2015-08-14 20:05:35.726000,2015-08-14 20:07:07.137000,2,1,1,0,0
1761593,999949_1,999949,2015-06-24 16:38:47.153000,2015-06-24 16:40:43.363000,3,0,3,0,0
1761609,999959_1,999959,2015-05-17 05:06:54.296000,2015-05-17 05:10:45.482000,2,1,1,0,0
1761648,999986_2,999986,2015-08-28 18:54:44.555000,2015-08-28 18:58:47.689000,2,1,1,0,0


31992
